In [1]:
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os

In [2]:

"""
some variables needed to be set up by users
"""
conference_url = "https://openreview.net/group?id=ICLR.cc/2022/Conference" # the conference url to download papers from
edgedriver_path = 'C:\\Users\\xiaoyang\\Downloads\\edgedriver_win64\msedgedriver.exe' # the chromedriver.exe path
root = r'E:\OneDrive\academics\papers\conferences\ICLR-2022-ALL'.replace('\\','/') # file path to save the downloaded papers

os.makedirs(root, exist_ok=True)
print(root)

E:/OneDrive/academics/papers/conferences/ICLR-2022-ALL


In [3]:
"""
conference urls examples

cvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)
eccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)
eccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)
iccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)
icml: http://proceedings.mlr.press/v119/ (ICML 2020)
icml: http://proceedings.mlr.press/v139/ (ICML 2021)
neurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)
iclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)
siggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)

"""




'\nconference urls examples\n\ncvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)\neccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)\neccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)\niccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)\nicml: http://proceedings.mlr.press/v119/ (ICML 2020)\nicml: http://proceedings.mlr.press/v139/ (ICML 2021)\nneurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)\niclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)\nsiggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)\n\n'

In [86]:
driver = webdriver.Edge(edgedriver_path)  
driver.get(conference_url)

In [15]:
driver.find_element_by_partial_link_text('Spotlight').click()

In [48]:
elementllist =  driver.find_elements_by_tag_name('h4')[1:]

In [81]:
# /html/body/div/div[3]/div/div/main/div/div[3]/div/div[2]/div[2]/nav/ul/li[5]
driver.find_element_by_xpath('/html/body/div/div[3]/div/div/main/div/div[3]/div/div[2]/div[2]/nav/ul/li[5]').text

'›'

In [85]:
driver.find_element_by_link_text('›').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"›"}
  (Session info: MicrosoftEdge=102.0.1245.41)


In [93]:
e = elementllist[50]

In [92]:
len(elementllist)

250

In [94]:
e.get_attribute('innerHTML')

WebDriverException: Message: chrome not reachable
  (Session info: MicrosoftEdge=102.0.1245.41)


In [95]:
e.find_elements_by_xpath('a')[1].get_attribute('href')

WebDriverException: Message: chrome not reachable
  (Session info: MicrosoftEdge=102.0.1245.41)


In [88]:
from selenium.common.exceptions import NoSuchElementException

def retrieve_from_iclr(driver):
    pdfurllist =  []
    pdfnamelist = []
    
    # three sections: oral, spotlight, poster
    for num_section, section in enumerate(['Oral', 'Spotlight', 'Poster']):
        driver.get(conference_url)
        _ = time.sleep(3)
        driver.find_element_by_partial_link_text(section).click()
        loop_pages = True
        while loop_pages:
            elementllist = driver.find_elements_by_tag_name('h4')[1:]
            for i, element in enumerate(elementllist): 
                pdfnamelist.append(elementllist[i].text)
                pdfurllist.append(elementllist[i].find_elements_by_xpath('a')[1].get_attribute('href'))
            try: 
                driver.find_element_by_link_text('›').click()
                _ = time.sleep(3)
            except NoSuchElementException as e:
                loop_pages = False


    return pdfurllist, pdfnamelist

print('Retrieving pdf urls. This could take some time...')
pdfurllist, pdfnamelist = retrieve_from_iclr(driver)



Retrieving pdf urls. This could take some time...


In [103]:
zip_pdf = []
for name, url in zip(pdfnamelist, pdfurllist):
    if name != '':
        zip_pdf.append((name, url))

In [107]:
pdfurllist =  []
pdfnamelist = []
for name, url in zip_pdf:
    pdfurllist.append(url)
    pdfnamelist.append(name)

In [108]:
# check the retrieved urls
print('The first 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[i])
print('\nThe last 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[-(i+1)])


The first 5 pdf urls:

https://openreview.net/pdf?id=FPCMqjI0jXN
https://openreview.net/pdf?id=NudBMY-tzDr
https://openreview.net/pdf?id=tYRrOdSnVUy
https://openreview.net/pdf?id=YWNAX0caEjI
https://openreview.net/pdf?id=uxgg9o7bI_3

The last 5 pdf urls:

https://openreview.net/pdf?id=AjGC97Aofee
https://openreview.net/pdf?id=YgPqNctmyd
https://openreview.net/pdf?id=j-63FSNcO5a
https://openreview.net/pdf?id=kezNJydWvE
https://openreview.net/pdf?id=rJvY_5OzoI


In [109]:
# check the retrieved paper titles
print('The first 5 pdf titles:\n')
for i in range(5):
    
    print(pdfnamelist[i])
print('\nThe last 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[-(i+1)])


The first 5 pdf titles:

Domino: Discovering Systematic Errors with Cross-Modal Embeddings
Natural Language Descriptions of Deep Visual Features
Non-Transferable Learning: A New Approach for Model Ownership Verification and Applicability Authorization
Neural Structured Prediction for Inductive Node Classification
A New Perspective on "How Graph Neural Networks Go Beyond Weisfeiler-Lehman?"

The last 5 pdf titles:

Learning Efficient Image Super-Resolution Networks via Structure-Regularized Pruning
Towards Building A Group-based Unsupervised Representation Disentanglement Framework
Learning Disentangled Representation by Exploiting Pretrained Generative Models: A Contrastive Learning View
Clean Images are Hard to Reblur: Exploiting the Ill-Posed Inverse Task for Dynamic Scene Deblurring
Multi-Critic Actor Learning: Teaching RL Policies to Act with Style


In [110]:
print('The number of papers is ', len(pdfnamelist))
assert len(pdfnamelist)==len(pdfurllist), 'The number of titles and the number of urls are not matched. \
                                            You might solve the problem by checking the HTML code in the \
                                            website yourself or you could ask the author by raising an issue.'

The number of papers is  1094


In [ ]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
for i, url in enumerate(pdfurllist):
    if url != None :      
        pdfname = slugify(pdfnamelist[i])
        if os.path.isfile(root+'/'+pdfname+".pdf"):
            print('existed', i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
        else:
            print(i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
            
            try_download = True
            while try_download:
                try:
                    data = requests.get(pdfurllist[i], timeout=80, headers=headers).content
                    try_download = False
                except TimeoutError as e:
                    _ = time.sleep(random.uniform(1,2)) # for anti-anti-crawler purpose
                except ConnectionError as e: 
                    _ = time.sleep(random.uniform(1,2)) # for anti-anti-crawler purpose
                
                    
            with open(root+'/'+pdfname+".pdf", 'wb')  as f:
                f.write(data)  